## Load and split data

In [1]:
import pandas as pd


dfa = pd.read_excel('Sampling_Data_Alcobaca.xlsx')
dfa = dfa.loc[~dfa['Sample'].str.lower().str.contains('tomates')]

### Water stations

In [2]:
water = dfa.loc[dfa['Sample'].str.lower().str.contains('water')]
water = water.assign(labels=['A1', 'A2', 'A3', 'A6', 'A4', 'A5'])
water = water.reset_index().set_index('labels').sort_index()

### Soil stations

In [3]:
soil = dfa.loc[dfa['Sample'].str.lower().str.contains('soil')]
soil = soil.drop('ALC022')  # Error in the analisys
soil = soil.drop('ALC011')  # Duplication of ALC010
soil = soil.assign(
    labels=['AL01', 'AL02', 'AL03', 'AL04', 'AL05', 'AL06', 'AL07', 'AL08', 'AL08',
            'AL09', 'AL10', 'AL11', 'AL12', 'AL13', 'AL14', 'AL15', 'AL16', 'AL17']
)
soil = soil.reset_index().set_index('labels').sort_index()

In [4]:
import pandas as pd


def fix_position(pos):
    pos = pos.strip()
    pos = pos.replace("'", "")
    pos = pos.replace(",", ".")
    pos = pos.replace("°", "")
    pos = list(map(float, pos.split()))
    pos = pos[0] + pos[1] / 60.0 + pos[2] / 60.0 /60.0
    return -pos


dfc = pd.read_excel('Sampling_Data_Caravelas.xlsx', )
dfc.rename(inplace=True,
    columns={
        'Amostras': 'samples',
        'Profundidade (m)': 'depth (m)',
        'hora': 'time',
        'Temperatura (°C)': 'temperature',
        'Salinidade': 'salinity',
    }
)

dfc['Latitude'] = dfc['Latitude'].apply(fix_position)
dfc['Longitude'] = dfc['Longitude'].apply(fix_position)

dfc.index +=1
dfc.index = 'S' + dfc.index.astype(str)

In [5]:
import folium
from folium.plugins import Fullscreen

m = folium.Map()

for idx, row in soil.iterrows():
    icon = folium.Icon(color='green')
    location = row['Latitude'], row['Longitude']
    popup = '{}'.format(idx)
    folium.Marker(location=location, icon=icon, popup=popup).add_to(m)

for idx, row in water.iterrows():
    icon = folium.Icon(color='blue')
    location = row['Latitude'], row['Longitude']
    popup = '{}'.format(idx)
    folium.Marker(location=location, icon=icon, popup=popup).add_to(m)

for idx, row in dfc.iterrows():
    icon = folium.Icon(color='orange')
    location = row['Latitude'], row['Longitude']
    popup = '{}'.format(idx)
    folium.Marker(location=location, icon=icon, popup=popup).add_to(m)

m.fit_bounds(m.get_bounds())

Fullscreen().add_to(m)

m